### Initialize

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='omkarsoak-ee')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## PROCESS GEE

In [ ]:
def export_change_detection_data(lon, lat, city, state, filename):
    # Define study area based on coordinates with hardcoded buffer size
    center = ee.Geometry.Point([lon, lat])
    geometry = center.buffer(2560).bounds()

    # Specify
    period2019 = {'start': '2019-07-01', 'end': '2019-09-30'}
    period2024 = {'start': '2024-07-01', 'end': '2024-09-30'}
    cloud_thresh = 3
    folder_name='UGACH'

    # Function to create classified composite
    def create_classified_composite(start_date, end_date):
        dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
            .filter(ee.Filter.date(start_date, end_date)) \
            .filter(ee.Filter.bounds(geometry))

        dw_composite = dw.select('label').reduce(ee.Reducer.mode())

        # Remap to custom classes:
        # 0: water, 1: built, 2: bare, 3: sparse-vegetation, 4: trees, 5: crops, 6:others
        return dw_composite.remap(
            [0, 1, 2, 3, 4, 5, 6, 7, 8],      # source values
            [0, 4, 3, 3, 5, 3, 1, 2, 6],    # destination values
            6                               # other values
        ).updateMask(ee.Image(1))

    # Create composites for each period
    composite2019 = create_classified_composite(period2019['start'], period2019['end'])
    composite2024 = create_classified_composite(period2024['start'], period2024['end'])

    # Define class names for reference (not used in export but useful for documentation)
    class_names = {
        0: 'water',
        1: 'built',
        2: 'bare',
        3: 'sparse',
        4: 'trees',
        5: 'crops',
        6: 'others'
    }

    # Load Sentinel-2 data for 2019 with hardcoded cloud threshold
    sentinel2019 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(geometry) \
        .filterDate(period2019['start'], period2019['end']) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_thresh)) \
        .median()

    # Load Sentinel-2 data for 2024 with hardcoded cloud threshold
    sentinel2024 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(geometry) \
        .filterDate(period2024['start'], period2024['end']) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_thresh)) \
        .median()

    # Create enhanced RGB composites
    enhanced_rgb2019 = sentinel2019.select('B4', 'B3', 'B2').multiply(2.0)
    enhanced_rgb2024 = sentinel2024.select('B4', 'B3', 'B2').multiply(2.0)

    # Define visualization parameters for RGB
    rgb_viz_params = {
        'min': 0,
        'max': 3000,
        'bands': ['B4', 'B3', 'B2'],
        'gamma' : 1
    }

    # Export 2019 Land Cover Classification
    mask_task_2019 = ee.batch.Export.image.toDrive(
        image=composite2019.clip(geometry).uint16(),
        description=f'm_{filename}_2019',
        region=geometry,
        fileFormat='GEOTIFF',
        crs='EPSG:3857',
        dimensions='512x512',
        folder=folder_name,
        fileNamePrefix=f'm_{filename}_2019'
    )

    # Export 2024 Land Cover Classification
    mask_task_2024 = ee.batch.Export.image.toDrive(
        image=composite2024.clip(geometry).uint16(),
        description=f'm_{filename}_2024',
        region=geometry,
        fileFormat='GEOTIFF',
        crs='EPSG:3857',
        dimensions='512x512',
        folder=folder_name,
        fileNamePrefix=f'm_{filename}_2024'
    )

    # Export 2019 Enhanced RGB
    rgb_task_2019 = ee.batch.Export.image.toDrive(
        image=enhanced_rgb2019.clip(geometry).visualize(**rgb_viz_params).uint16(),
        description=f'{filename}_2019',
        region=geometry,
        fileFormat='GEOTIFF',
        crs='EPSG:3857',
        dimensions='512x512',
        folder=folder_name,
        fileNamePrefix=f'{filename}_2019'
    )

    # Export 2024 Enhanced RGB
    rgb_task_2024 = ee.batch.Export.image.toDrive(
        image=enhanced_rgb2024.clip(geometry).visualize(**rgb_viz_params).uint16(),
        description=f'{filename}_2024',
        region=geometry,
        fileFormat='GEOTIFF',
        crs='EPSG:3857',
        dimensions='512x512',
        folder=folder_name,
        fileNamePrefix=f'{filename}_2024'
    )

    # Start the export tasks
    mask_task_2019.start()
    mask_task_2024.start()
    rgb_task_2019.start()
    rgb_task_2024.start()

    print(f"Export tasks have been started for {filename}.")

    # # Return tasks for reference
    # return {
    #     'mask_2019': mask_task_2019,
    #     'mask_2024': mask_task_2024,
    #     'rgb_2019': rgb_task_2019,
    #     'rgb_2024': rgb_task_2024
    # }



## RUN

### Import Data

In [ ]:
import io
import pandas as pd
data = pd.read_csv(io.StringIO('''
Netherlands,Amsterdam,2019,2024,"4.85965202286674,52.3659617416252",52.36596174,4.859652023,"4.85965202286674,52.4159617416252","4.90965202286674,52.4159617416252","4.90965202286674,52.3659617416252","4.90965202286674,52.3159617416252","4.85965202286674,52.3159617416252","4.80965202286674,52.3159617416252","4.80965202286674,52.3659617416252","4.80965202286674,52.4159617416252"
'''), header=None)


### Run Process

In [ ]:
import os

coord_list = [4, 7, 8, 9, 10, 11, 12, 13, 14]
coord_name = ['C', 'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
ncities = data.shape[0]

# Ensure the error log file exists
if not os.path.exists("error_log.txt"):
    open("error_log.txt", "w").close()

error_log = open("error_log.txt", "a")

# Process each city
for j in range(ncities):
    cityData = data.iloc[j]
    city = cityData[1]
    state = cityData[0]
    print(city, state)

    for i in range(9):
        try:
            coord = cityData.iloc[coord_list[i]].split(',')
            filename = f'{state}_{city}_{coord_name[i]}'
            print(filename)

            lon = float(coord[0])
            lat = float(coord[1])
            export_change_detection_data(lon, lat, city, state, filename)

        except Exception as e:
            error_message = f"Error processing {state}, {city}, {coord_name[i]}: {e}\n"
            print(error_message.strip())
            error_log.write(f"{filename}\n")
    print()

error_log.close()
print("All tasks have been started.")


Amsterdam Netherlands
Netherlands_Amsterdam_C
Export tasks have been started for Netherlands_Amsterdam_C.
Netherlands_Amsterdam_N
Export tasks have been started for Netherlands_Amsterdam_N.
Netherlands_Amsterdam_NE
Export tasks have been started for Netherlands_Amsterdam_NE.
Netherlands_Amsterdam_E
Export tasks have been started for Netherlands_Amsterdam_E.
Netherlands_Amsterdam_SE
Export tasks have been started for Netherlands_Amsterdam_SE.
Netherlands_Amsterdam_S
Export tasks have been started for Netherlands_Amsterdam_S.
Netherlands_Amsterdam_SW
Export tasks have been started for Netherlands_Amsterdam_SW.
Netherlands_Amsterdam_W
Export tasks have been started for Netherlands_Amsterdam_W.
Netherlands_Amsterdam_NW
Export tasks have been started for Netherlands_Amsterdam_NW.

All tasks have been started.
